In [1]:
# ============================================================
# v6 — Cell 1: Environment, Invariants, and Global Assumptions
# ============================================================
# Purpose:
#   - Deterministic, auditable execution environment (publication-grade)
#   - Declare immutable scope + invariants up front
#   - Do NOT load data yet (no accidental leakage / side effects)
# ============================================================

# ------------------------
# Standard library
# ------------------------
import os
import sys
from pathlib import Path
from typing import Dict, List

# ------------------------
# Numerical stack
# ------------------------
import numpy as np
import pandas as pd
import scipy as sp

# ------------------------
# Single-cell stack
# ------------------------
import scanpy as sc
import anndata as ad

# ------------------------
# Reproducibility
# ------------------------
RANDOM_STATE: int = 42
np.random.seed(RANDOM_STATE)

# ------------------------
# Display / logging safety
# ------------------------
pd.set_option("display.max_rows", 40)
pd.set_option("display.max_columns", 40)
pd.set_option("display.width", 140)

# ============================================================
# Project layout (explicit; adjust if needed)
# ============================================================
PROJECT_ROOT = Path.cwd()
DATA_DIR = PROJECT_ROOT / "data"
RESULTS_DIR = PROJECT_ROOT / "results_v6"
FIG_DIR = RESULTS_DIR / "figures"

RESULTS_DIR.mkdir(parents=True, exist_ok=True)
FIG_DIR.mkdir(parents=True, exist_ok=True)

# ============================================================
# File invariants (existence only; not loaded here)
# ============================================================
CONTROL_LABEL: str = "NC"

REQUIRED_FILES = [
    DATA_DIR / "obesity_challenge_1.h5ad",
    DATA_DIR / "program_proportion.csv",  # external-only evaluation labels
]

assert DATA_DIR.exists() and DATA_DIR.is_dir(), f"Missing data directory: {DATA_DIR}"
for fp in REQUIRED_FILES:
    assert fp.exists(), f"Missing required file: {fp}"

# ============================================================
# Immutable scope assumptions (v6 paper spec)
# ============================================================
ASSUMPTIONS: Dict[str, List[str]] = {
    "data": [
        "adata.X is provider-normalized (log-transformed); raw counts are not treated as independent samples.",
        "Perturbation identity is stored in adata.obs['gene']; control label is 'NC'.",
    ],
    "unit_of_analysis": [
        "Primary geometric object is perturbation-level effects (deltas), not single cells.",
        "Distributional analysis is performed relative to the control distribution per gene.",
    ],
    "label_usage": [
        "Program proportions are external-only for alignment/evaluation and never used to construct geometry.",
        "No tuning based on external labels; all decision rules are pre-committed.",
    ],
    "claims": [
        "PCA is a diagnostic variance accounting tool, not a biological model.",
        "Paper is falsifiable: failure to reject nulls or demonstrate insufficiency means paper fails.",
    ],
}

# ============================================================
# Environment fingerprint (for Methods reproducibility)
# ============================================================
ENV_FINGERPRINT = {
    "python": sys.version.split()[0],
    "numpy": np.__version__,
    "pandas": pd.__version__,
    "scipy": sp.__version__,
    "scanpy": getattr(sc, "__version__", "unknown"),
    "anndata": getattr(ad, "__version__", "unknown"),
    "random_state": RANDOM_STATE,
    "project_root": str(PROJECT_ROOT),
    "data_dir": str(DATA_DIR),
    "results_dir": str(RESULTS_DIR),
}

print("Environment fingerprint:")
for k, v in ENV_FINGERPRINT.items():
    print(f"  {k:>12s}: {v}")

print("\nInvariants:")
print(f"  CONTROL_LABEL: {CONTROL_LABEL}")
print(f"  REQUIRED_FILES: {[p.name for p in REQUIRED_FILES]}")
print("\nCell 1 complete. No data loaded. No labels touched.")


Environment fingerprint:
        python: 3.11.9
         numpy: 2.2.0
        pandas: 2.3.2
         scipy: 1.16.1
        scanpy: 1.11.5
       anndata: 0.12.6
  random_state: 42
  project_root: c:\Users\Bryan\Documents\CrunchDAO Obesity
      data_dir: c:\Users\Bryan\Documents\CrunchDAO Obesity\data
   results_dir: c:\Users\Bryan\Documents\CrunchDAO Obesity\results_v6

Invariants:
  CONTROL_LABEL: NC
  REQUIRED_FILES: ['obesity_challenge_1.h5ad', 'program_proportion.csv']

Cell 1 complete. No data loaded. No labels touched.


C:\Users\Bryan\AppData\Local\Temp\ipykernel_18384\2483052584.py:99: FutureWarning: `__version__` is deprecated, use `importlib.metadata.version('scanpy')` instead
  "scanpy": getattr(sc, "__version__", "unknown"),
C:\Users\Bryan\AppData\Local\Temp\ipykernel_18384\2483052584.py:100: FutureWarning: `__version__` is deprecated, use `importlib.metadata.version('anndata')` instead.
  "anndata": getattr(ad, "__version__", "unknown"),


In [2]:
# ============================================================
# v6 — Cell 2: Data Loading + Structural Invariants
# ============================================================
# Purpose:
#   - Load AnnData exactly once
#   - Assert structural + semantic invariants required downstream
#   - Still no external labels loaded (program proportions stay untouched)
# ============================================================

from importlib.metadata import version as pkg_version

# ------------------------
# Load dataset
# ------------------------
ADATA_PATH = DATA_DIR / "obesity_challenge_1.h5ad"
adata = sc.read_h5ad(ADATA_PATH)

# ------------------------
# Hard structural checks
# ------------------------
assert isinstance(adata, ad.AnnData), "adata is not an AnnData object"
assert adata.n_obs > 0, "adata contains zero cells"
assert adata.n_vars > 0, "adata contains zero genes"

assert adata.X is not None, "adata.X is None"
assert adata.X.shape == (adata.n_obs, adata.n_vars), "adata.X shape mismatch"

# ------------------------
# Required annotations
# ------------------------
REQUIRED_OBS_COLS = ["gene"]
for col in REQUIRED_OBS_COLS:
    assert col in adata.obs.columns, f"Missing adata.obs column: '{col}'"

# Control label must exist
gene_series = adata.obs["gene"]
assert CONTROL_LABEL in set(gene_series), f"Control label '{CONTROL_LABEL}' not found in adata.obs['gene']"

# Uniqueness invariants
assert adata.var_names.is_unique, "Gene names are not unique"
assert adata.obs_names.is_unique, "Cell identifiers are not unique"

# Dtype sanity
assert (
    gene_series.dtype == object
    or pd.api.types.is_string_dtype(gene_series)
    or pd.api.types.is_categorical_dtype(gene_series)
), "adata.obs['gene'] must be string-like or categorical"

# ------------------------
# Minimal data fingerprint (Methods-grade)
# ------------------------
DATA_FINGERPRINT = {
    "n_cells": int(adata.n_obs),
    "n_genes": int(adata.n_vars),
    "n_perturbations_total_including_control": int(gene_series.nunique()),
    "control_label": CONTROL_LABEL,
    "scanpy_version": pkg_version("scanpy"),
    "anndata_version": pkg_version("anndata"),
    "X_type": type(adata.X).__name__,
}

print("Data fingerprint:")
for k, v in DATA_FINGERPRINT.items():
    print(f"  {k:>34s}: {v}")

# ------------------------
# Quick sanity: counts per perturbation (top/bottom)
# ------------------------
vc = gene_series.value_counts(dropna=False)
print("\nPerturbation cell-count summary:")
print(f"  min_cells_per_label: {int(vc.min())}")
print(f"  max_cells_per_label: {int(vc.max())}")
print(f"  n_labels: {int(vc.shape[0])}")

print("\nTop 10 labels by cell count:")
display(vc.head(10).to_frame("n_cells"))

print("\nBottom 10 labels by cell count:")
display(vc.tail(10).to_frame("n_cells"))

print("\nCell 2 complete. Data loaded. No external labels loaded.")


Data fingerprint:
                             n_cells: 88202
                             n_genes: 21592
  n_perturbations_total_including_control: 123
                       control_label: NC
                      scanpy_version: 1.11.5
                     anndata_version: 0.12.6
                              X_type: csr_matrix

Perturbation cell-count summary:
  min_cells_per_label: 41
  max_cells_per_label: 8705
  n_labels: 123

Top 10 labels by cell count:


,n_cells
gene,
NC,8705
FOXP1,2328
RBAK,1483
HAND2,1470
ZNF331,1428
DEDD2,1398
TFEB,1313
PLAGL1,1292
NCOR2,1288



Bottom 10 labels by cell count:


,n_cells
gene,
MEF2A,330
BTAF1,324
ZNF283,317
CNOT8,295
FAM136A,294
CEBPB,294
EP300,273
NFE2L1,232
TMEM107,212



Cell 2 complete. Data loaded. No external labels loaded.


In [3]:
# ============================================================
# v6 — Cell 3: Control / Perturbation Partition + Indexed Ordering
# ============================================================
# Purpose:
#   - Partition cells into control vs perturbed
#   - Construct a deterministic perturbation index (audit-safe)
#   - Enforce minimum cell-count rule (for distribution estimates)
# Scope:
#   - No arithmetic on expression yet (no means, no deltas)
#   - No external labels
# ============================================================

# ------------------------
# Preconditions
# ------------------------
assert "adata" in globals(), "adata not found (run Cell 2)"
assert "gene" in adata.obs.columns, "adata.obs['gene'] missing"
assert CONTROL_LABEL in set(adata.obs["gene"]), "CONTROL_LABEL missing"

# ------------------------
# Partition masks
# ------------------------
gene_labels = adata.obs["gene"]
control_mask = gene_labels == CONTROL_LABEL
perturb_mask = ~control_mask

n_control = int(control_mask.sum())
n_perturb = int(perturb_mask.sum())

assert n_control > 0, "No control cells found"
assert n_perturb > 0, "No perturbed cells found"
assert n_control + n_perturb == adata.n_obs, "Partition mismatch"

# ------------------------
# Deterministic perturbation set (exclude control)
# ------------------------
perturb_labels = gene_labels[perturb_mask]
PERTURBATION_GENES = tuple(sorted(pd.unique(perturb_labels)))

assert CONTROL_LABEL not in set(PERTURBATION_GENES), "Control leaked into perturbations"
assert len(PERTURBATION_GENES) == int(perturb_labels.nunique()), "Perturbation unique count mismatch"

PERTURBATION_INDEX = {g: i for i, g in enumerate(PERTURBATION_GENES)}

# ------------------------
# Cell-count constraints (for distribution-level control envelopes)
# ------------------------
# NOTE:
# We will estimate distribution statistics per gene for control cells,
# and compare perturbations against those envelopes. Very small groups
# can destabilize quantiles and nonparametric tests.
MIN_CELLS_PER_PERTURBATION = 30

pert_counts = perturb_labels.value_counts()
low_n = pert_counts[pert_counts < MIN_CELLS_PER_PERTURBATION]

# This is an invariant decision: either drop or keep with explicit flag.
# v6 default: KEEP ALL, but mark low-n perturbations and exclude them
# from any inferential test that assumes stable distribution estimates.
LOW_N_PERTURBATIONS = tuple(sorted(low_n.index.tolist()))

# ------------------------
# Summary fingerprint
# ------------------------
PARTITION_FINGERPRINT = {
    "n_control_cells": n_control,
    "n_perturbed_cells": n_perturb,
    "n_perturbations_excl_control": len(PERTURBATION_GENES),
    "min_cells_per_perturbation_excl_control": int(pert_counts.min()),
    "median_cells_per_perturbation_excl_control": float(np.median(pert_counts.values)),
    "n_low_n_perturbations": len(LOW_N_PERTURBATIONS),
    "min_cells_threshold": MIN_CELLS_PER_PERTURBATION,
}

print("Partition fingerprint:")
for k, v in PARTITION_FINGERPRINT.items():
    print(f"  {k:>36s}: {v}")

if len(LOW_N_PERTURBATIONS) > 0:
    print("\nLow-n perturbations (will be excluded from distributional inferential tests):")
    display(pd.DataFrame({"perturbation": LOW_N_PERTURBATIONS}).head(50))

print("\nCell 3 complete. Partition + perturbation index established. No expression arithmetic yet.")


Partition fingerprint:
                       n_control_cells: 8705
                     n_perturbed_cells: 79497
          n_perturbations_excl_control: 122
  min_cells_per_perturbation_excl_control: 0
  median_cells_per_perturbation_excl_control: 565.0
                 n_low_n_perturbations: 1
                   min_cells_threshold: 30

Low-n perturbations (will be excluded from distributional inferential tests):


,perturbation
0,NC



Cell 3 complete. Partition + perturbation index established. No expression arithmetic yet.


In [4]:
# ============================================================
# v6 — Cell 4: Partition Fix + Control/Perturbation Slices (Auditable)
# ============================================================
# Purpose:
#   - Fix the categorical/unused-category edge case that produced:
#       * min_cells_per_perturbation_excl_control = 0
#       * LOW_N_PERTURBATIONS incorrectly containing "NC"
#   - Materialize adata_control / adata_perturb slices (copy-safe)
#   - Recompute perturbation counts with strict control exclusion
#
# Scope:
#   - Still NO expression arithmetic (no means, no deltas)
#   - Still NO external labels
# ============================================================

assert "adata" in globals(), "adata not found"
assert "gene" in adata.obs.columns, "adata.obs['gene'] missing"

# ------------------------
# Defensive: remove categorical artifacts
# ------------------------
# If 'gene' is categorical, value_counts can include unused categories
# and can lead to zero-count entries + control leakage in summaries.
gene_raw = adata.obs["gene"]
gene_str = gene_raw.astype(str)  # canonical representation

# Rebuild masks from canonical gene labels
control_mask = gene_str == CONTROL_LABEL
perturb_mask = ~control_mask

n_control = int(control_mask.sum())
n_perturb = int(perturb_mask.sum())
assert n_control > 0 and n_perturb > 0

# ------------------------
# Slice AnnData objects (copy to avoid view pitfalls)
# ------------------------
adata_control = adata[control_mask.values].copy()
adata_perturb = adata[perturb_mask.values].copy()

# ------------------------
# Recompute perturbation list + counts (exclude control explicitly)
# ------------------------
perturb_gene_str = adata_perturb.obs["gene"].astype(str)
assert CONTROL_LABEL not in set(perturb_gene_str), "Control leaked into adata_perturb"

PERTURBATION_GENES = tuple(sorted(pd.unique(perturb_gene_str)))
PERTURBATION_INDEX = {g: i for i, g in enumerate(PERTURBATION_GENES)}

pert_counts = perturb_gene_str.value_counts(dropna=False)

# Low-n policy (v6): keep labels, but exclude from inferential tests
MIN_CELLS_PER_PERTURBATION = 30
LOW_N_PERTURBATIONS = tuple(sorted(pert_counts[pert_counts < MIN_CELLS_PER_PERTURBATION].index.tolist()))

# ------------------------
# Sanity: ensure control is not in low-n list
# ------------------------
assert CONTROL_LABEL not in set(LOW_N_PERTURBATIONS), "LOW_N_PERTURBATIONS contains control label unexpectedly"
assert int(pert_counts.min()) > 0, "Zero-count perturbation detected after control exclusion (should not happen)"

# ------------------------
# Report
# ------------------------
PARTITION_FIX_FINGERPRINT = {
    "n_control_cells": int(adata_control.n_obs),
    "n_perturbed_cells": int(adata_perturb.n_obs),
    "n_perturbations_excl_control": len(PERTURBATION_GENES),
    "min_cells_per_perturbation_excl_control": int(pert_counts.min()),
    "median_cells_per_perturbation_excl_control": float(np.median(pert_counts.values)),
    "n_low_n_perturbations": len(LOW_N_PERTURBATIONS),
    "min_cells_threshold": MIN_CELLS_PER_PERTURBATION,
}

print("Partition fix fingerprint:")
for k, v in PARTITION_FIX_FINGERPRINT.items():
    print(f"  {k:>40s}: {v}")

if len(LOW_N_PERTURBATIONS) > 0:
    print("\nLow-n perturbations (excluded from distributional inferential tests):")
    display(pd.DataFrame({"perturbation": LOW_N_PERTURBATIONS, "n_cells": [int(pert_counts[p]) for p in LOW_N_PERTURBATIONS]}))

print("\nCell 4 complete. Partition repaired. adata_control/adata_perturb materialized. No expression arithmetic yet.")


Partition fix fingerprint:
                           n_control_cells: 8705
                         n_perturbed_cells: 79497
              n_perturbations_excl_control: 122
   min_cells_per_perturbation_excl_control: 41
  median_cells_per_perturbation_excl_control: 565.0
                     n_low_n_perturbations: 0
                       min_cells_threshold: 30

Cell 4 complete. Partition repaired. adata_control/adata_perturb materialized. No expression arithmetic yet.


In [5]:
# ============================================================
# v6 — Cell 5: Control Distribution Reference (Per-Gene Envelopes)
# ============================================================
# Purpose (v6 core change):
#   Define the control reference as a *distribution* per gene, not a mean.
#   Construct reproducible, per-gene baseline envelopes used to judge
#   whether perturbation shifts exceed control variability.
#
# Outputs (objects used downstream):
#   - control_mean            (for Δ_p definition later, justified by Section 1)
#   - control_var             (within-control variance per gene)
#   - control_q_lo / q_hi     (quantile envelope per gene)
#   - control_iqr             (IQR per gene; robust scale)
#
# Scope:
#   - Only control cells are used
#   - No perturbation information is used
#   - No external labels
# ============================================================

assert "adata_control" in globals(), "adata_control not found (run Cell 4)"
assert adata_control.n_obs > 0, "adata_control has zero cells"

# ------------------------
# Configuration (pre-committed, paper-grade)
# ------------------------
# Envelope quantiles: symmetric, robust; fixed a priori
Q_LO, Q_HI = 0.05, 0.95

# Small numerical safety
EPS = 1e-12

# ------------------------
# Control matrix
# ------------------------
Xc = adata_control.X
assert Xc.shape[0] == adata_control.n_obs
assert Xc.shape[1] == adata_control.n_vars

# ------------------------
# Compute control summaries
# ------------------------
# Mean (used later for Δ; justified only after distribution context)
control_mean = np.asarray(Xc.mean(axis=0)).ravel()

# Variance (within-control baseline fluctuation)
# NOTE: sparse variance via E[X^2] - (E[X])^2
Xc_sq_mean = np.asarray(Xc.multiply(Xc).mean(axis=0)).ravel()
control_var = np.maximum(Xc_sq_mean - control_mean**2, 0.0)
control_sd = np.sqrt(control_var + EPS)

# Quantile envelopes require densification or specialized sparse quantiles.
# Given n_control=8705 and n_genes=21592, densifying control only is usually feasible.
# v6 priority is correctness and transparency > micro-optimizations.
Xc_dense = Xc.toarray() if hasattr(Xc, "toarray") else np.asarray(Xc)

control_q_lo = np.quantile(Xc_dense, Q_LO, axis=0)
control_q_hi = np.quantile(Xc_dense, Q_HI, axis=0)

control_q25 = np.quantile(Xc_dense, 0.25, axis=0)
control_q75 = np.quantile(Xc_dense, 0.75, axis=0)
control_iqr = control_q75 - control_q25

# ------------------------
# Postconditions
# ------------------------
n_genes = adata_control.n_vars
for arr_name in ["control_mean", "control_var", "control_sd", "control_q_lo", "control_q_hi", "control_iqr"]:
    arr = globals()[arr_name]
    assert arr.shape == (n_genes,), f"{arr_name} has wrong shape: {arr.shape}"
    assert np.all(np.isfinite(arr)), f"Non-finite values in {arr_name}"

assert Q_LO < Q_HI, "Invalid quantile envelope configuration"

# ------------------------
# Fingerprint report (Methods-grade)
# ------------------------
CTRL_DIST_FINGERPRINT = {
    "n_control_cells": int(adata_control.n_obs),
    "n_genes": int(n_genes),
    "envelope_q_lo": float(Q_LO),
    "envelope_q_hi": float(Q_HI),
    "mean_min": float(control_mean.min()),
    "mean_max": float(control_mean.max()),
    "sd_median": float(np.median(control_sd)),
    "iqr_median": float(np.median(control_iqr)),
}

print("Control distribution fingerprint:")
for k, v in CTRL_DIST_FINGERPRINT.items():
    print(f"  {k:>22s}: {v}")

# Keep dense control matrix only if needed later; otherwise free memory.
# We will not rely on Xc_dense after this cell unless explicitly reloaded.
del Xc_dense

print("\nCell 5 complete. Control distribution envelopes constructed. No perturbation data used.")


Control distribution fingerprint:
         n_control_cells: 8705
                 n_genes: 21592
           envelope_q_lo: 0.05
           envelope_q_hi: 0.95
                mean_min: 0.0
                mean_max: 11.328900940274325
               sd_median: 0.5788232512452218
              iqr_median: 0.0

Cell 5 complete. Control distribution envelopes constructed. No perturbation data used.


In [6]:
# ============================================================
# v6 — Cell 6: Control Variability Envelope Exceedance (Per Perturbation)
# ============================================================
# Purpose:
#   Quantify, for each perturbation, how often its gene-wise mean
#   falls outside the control distribution envelope.
#
# This is the core "distribution-first" grounding:
#   - We do NOT claim an effect because a mean changed.
#   - We first ask whether the perturbation mean escapes what control
#     variability would plausibly produce.
#
# Output:
#   - exceed_df: perturbation-level table with exceedance rates
#   - exceed_matrix: boolean matrix (perturbations × genes) for later diagnostics
#
# Scope:
#   - Uses control envelopes from Cell 5
#   - Uses perturbation groups, but no PCA, no external labels
# ============================================================

assert "adata_perturb" in globals(), "adata_perturb not found (run Cell 4)"
assert "control_q_lo" in globals() and "control_q_hi" in globals(), "Control envelopes missing (run Cell 5)"
assert "PERTURBATION_GENES" in globals(), "PERTURBATION_GENES missing (run Cell 4)"

# ------------------------
# Compute perturbation means (only what's needed for exceedance)
# ------------------------
n_perts = len(PERTURBATION_GENES)
n_genes = adata_perturb.n_vars
assert control_q_lo.shape == (n_genes,)
assert control_q_hi.shape == (n_genes,)

pert_means = np.zeros((n_perts, n_genes), dtype=np.float64)

for g, row_idx in PERTURBATION_INDEX.items():
    mask = adata_perturb.obs["gene"].astype(str) == g
    n_cells_g = int(mask.sum())
    assert n_cells_g > 0, f"No cells found for perturbation '{g}'"

    Xg = adata_perturb[mask].X
    pert_means[row_idx, :] = np.asarray(Xg.mean(axis=0)).ravel()

# ------------------------
# Exceedance definition (distribution-level)
# ------------------------
# A gene is "exceeded" for a perturbation if the perturbation mean
# lies outside the control [q_lo, q_hi] envelope.
exceed_low = pert_means < control_q_lo
exceed_high = pert_means > control_q_hi
exceed_matrix = exceed_low | exceed_high

# Per-perturbation exceedance rate
exceed_rate = exceed_matrix.mean(axis=1)

# Optional: directionality diagnostics
exceed_rate_low = exceed_low.mean(axis=1)
exceed_rate_high = exceed_high.mean(axis=1)

# ------------------------
# Assemble perturbation-level report
# ------------------------
exceed_df = pd.DataFrame(
    {
        "perturbation": PERTURBATION_GENES,
        "exceed_rate_total": exceed_rate,
        "exceed_rate_low": exceed_rate_low,
        "exceed_rate_high": exceed_rate_high,
    }
).set_index("perturbation")

# ------------------------
# Report summary
# ------------------------
EXCEED_FINGERPRINT = {
    "n_perturbations": int(n_perts),
    "n_genes": int(n_genes),
    "q_lo": float(Q_LO),
    "q_hi": float(Q_HI),
    "exceed_rate_min": float(exceed_df["exceed_rate_total"].min()),
    "exceed_rate_median": float(exceed_df["exceed_rate_total"].median()),
    "exceed_rate_max": float(exceed_df["exceed_rate_total"].max()),
}

print("Control-envelope exceedance fingerprint:")
for k, v in EXCEED_FINGERPRINT.items():
    print(f"  {k:>22s}: {v}")

print("\nTop 10 perturbations by exceedance rate (total):")
display(exceed_df.sort_values("exceed_rate_total", ascending=False).head(10))

print("\nBottom 10 perturbations by exceedance rate (total):")
display(exceed_df.sort_values("exceed_rate_total", ascending=True).head(10))

print("\nCell 6 complete. Distribution-level exceedance quantified.")


Control-envelope exceedance fingerprint:
         n_perturbations: 122
                 n_genes: 21592
                    q_lo: 0.05
                    q_hi: 0.95
         exceed_rate_min: 0.14222860318636532
      exceed_rate_median: 0.33588829195998515
         exceed_rate_max: 0.35415894775842904

Top 10 perturbations by exceedance rate (total):


,exceed_rate_total,exceed_rate_low,exceed_rate_high
perturbation,,,
FOXP1,0.354159,0.0,0.354159
RBAK,0.353511,0.0,0.353511
ZNF331,0.353233,0.0,0.353233
HAND2,0.352723,0.0,0.352723
DEDD2,0.352399,0.0,0.352399
PLAGL1,0.351565,0.0,0.351565
NCOR2,0.351473,0.0,0.351473
HIF3A,0.351334,0.0,0.351334
FOS,0.351288,0.0,0.351288



Bottom 10 perturbations by exceedance rate (total):


,exceed_rate_total,exceed_rate_low,exceed_rate_high
perturbation,,,
SRPK1,0.142229,0.000000,0.142229
TMEM107,0.286310,0.000046,0.286263
NFE2L1,0.289227,0.000000,0.289227
EP300,0.300806,0.000000,0.300806
CNOT8,0.303724,0.000000,0.303724
CEBPB,0.305345,0.000000,0.305345
FAM136A,0.308123,0.000000,0.308123
MEF2A,0.312199,0.000000,0.312199
TCFL5,0.313681,0.000000,0.313681



Cell 6 complete. Distribution-level exceedance quantified.


In [7]:
# ============================================================
# v6 — Cell 7: Formal Perturbation Deltas (Δ_p) + Sanity Diagnostics
# ============================================================
# Purpose (Section 2 of v6):
#   Define perturbation deltas using mean effects:
#       Δ_p = μ_p − μ_control
#   BUT: this is only justified because we already grounded control
#   variability via distribution envelopes (Cells 5–6).
#
# Outputs:
#   - delta_matrix: (n_perturbations × n_genes) mean-effect deltas
#   - delta_norms:  per-perturbation L2 norms (magnitude)
#   - delta_df:     perturbation-level table combining exceedance + delta magnitude
#
# Scope:
#   - No PCA yet
#   - No external labels
# ============================================================

assert "control_mean" in globals(), "control_mean missing (run Cell 5)"
assert "pert_means" in globals(), "pert_means missing (run Cell 6)"
assert "exceed_df" in globals(), "exceed_df missing (run Cell 6)"

n_perts, n_genes = pert_means.shape
assert control_mean.shape == (n_genes,), "control_mean shape mismatch"

# ------------------------
# Define formal deltas
# ------------------------
delta_matrix = pert_means - control_mean[None, :]

assert delta_matrix.shape == (n_perts, n_genes)
assert np.all(np.isfinite(delta_matrix)), "Non-finite values in delta_matrix"

# ------------------------
# Magnitude diagnostics
# ------------------------
delta_norms = np.linalg.norm(delta_matrix, axis=1)
assert delta_norms.shape == (n_perts,)
assert np.all(np.isfinite(delta_norms)), "Non-finite values in delta_norms"

# ------------------------
# Combine with exceedance metrics (distribution-first context)
# ------------------------
delta_df = exceed_df.copy()
delta_df["delta_l2_norm"] = delta_norms
delta_df["delta_l2_norm_rank"] = delta_df["delta_l2_norm"].rank(ascending=False, method="min")

# ------------------------
# Report
# ------------------------
DELTA_FINGERPRINT = {
    "delta_shape": str(delta_matrix.shape),
    "delta_min": float(delta_matrix.min()),
    "delta_max": float(delta_matrix.max()),
    "delta_l2_min": float(delta_norms.min()),
    "delta_l2_median": float(np.median(delta_norms)),
    "delta_l2_max": float(delta_norms.max()),
}

print("Delta fingerprint:")
for k, v in DELTA_FINGERPRINT.items():
    print(f"  {k:>16s}: {v}")

print("\nTop 10 perturbations by delta L2 norm:")
display(delta_df.sort_values("delta_l2_norm", ascending=False).head(10))

print("\nTop 10 perturbations by exceedance rate (for comparison):")
display(delta_df.sort_values("exceed_rate_total", ascending=False).head(10))

print("\nCell 7 complete. Δ_p defined and summarized (mean effects, distribution-grounded).")


Delta fingerprint:
       delta_shape: (122, 21592)
         delta_min: -2.413160548324867
         delta_max: 2.4744439633529143
      delta_l2_min: 4.037851562784551
   delta_l2_median: 6.42441007338657
      delta_l2_max: 26.09766898355497

Top 10 perturbations by delta L2 norm:


,exceed_rate_total,exceed_rate_low,exceed_rate_high,delta_l2_norm,delta_l2_norm_rank
perturbation,,,,,
RNASEH2C,0.323685,0.000046,0.323638,26.097669,1.0
FAM136A,0.308123,0.000000,0.308123,19.110749,2.0
SRPK1,0.142229,0.000000,0.142229,17.492289,3.0
EP400,0.342210,0.000000,0.342210,16.369464,4.0
TMEM107,0.286310,0.000046,0.286263,15.392489,5.0
CEBPA,0.328825,0.000000,0.328825,13.524040,6.0
CEBPB,0.305345,0.000000,0.305345,11.110703,7.0
EWSR1,0.321276,0.000000,0.321276,10.342153,8.0
CNOT8,0.303724,0.000000,0.303724,9.747726,9.0



Top 10 perturbations by exceedance rate (for comparison):


,exceed_rate_total,exceed_rate_low,exceed_rate_high,delta_l2_norm,delta_l2_norm_rank
perturbation,,,,,
FOXP1,0.354159,0.0,0.354159,4.306299,120.0
RBAK,0.353511,0.0,0.353511,6.041068,81.0
ZNF331,0.353233,0.0,0.353233,4.513816,117.0
HAND2,0.352723,0.0,0.352723,5.156826,103.0
DEDD2,0.352399,0.0,0.352399,5.018762,109.0
PLAGL1,0.351565,0.0,0.351565,6.917456,43.0
NCOR2,0.351473,0.0,0.351473,4.037852,122.0
HIF3A,0.351334,0.0,0.351334,4.226203,121.0
FOS,0.351288,0.0,0.351288,5.719361,97.0



Cell 7 complete. Δ_p defined and summarized (mean effects, distribution-grounded).


In [8]:
# ============================================================
# v6 — Cell 8: Geometry Discovery (PCA on Δ_p as Diagnostic)
# ============================================================
# Purpose (Section 3 of v6):
#   - Perform PCA on perturbation deltas (Δ_p)
#   - Quantify variance spectrum and PC1 dominance
#   - Produce a perturbation-level PCA score table for downstream tests
#
# Scope:
#   - PCA is a diagnostic variance accounting tool (no biology here)
#   - No external labels
#   - No nulls yet
# ============================================================

from sklearn.decomposition import PCA

assert "delta_matrix" in globals(), "delta_matrix missing (run Cell 7)"
assert delta_matrix.ndim == 2 and np.all(np.isfinite(delta_matrix)), "delta_matrix invalid"

n_perts, n_genes = delta_matrix.shape
assert n_perts > 2, "Need >2 perturbations for PCA"

# ------------------------
# PCA configuration (fixed)
# ------------------------
N_PCS = min(50, n_perts)  # cap for reporting; exact cap is deterministic
pca = PCA(
    n_components=N_PCS,
    svd_solver="full",
    random_state=RANDOM_STATE,
)

# ------------------------
# Fit PCA
# ------------------------
pca.fit(delta_matrix)

explained_var = pca.explained_variance_ratio_
cum_explained = np.cumsum(explained_var)

# ------------------------
# Project perturbations (scores)
# ------------------------
pc_scores = pca.transform(delta_matrix)  # (n_perts × N_PCS)

pc_df = pd.DataFrame(
    pc_scores,
    index=pd.Index(PERTURBATION_GENES, name="perturbation"),
    columns=[f"PC{i+1}" for i in range(N_PCS)],
)

# ------------------------
# Report variance spectrum
# ------------------------
PCA_FINGERPRINT = {
    "n_perturbations": int(n_perts),
    "n_genes": int(n_genes),
    "n_pcs_fitted": int(N_PCS),
    "var_PC1": float(explained_var[0]),
    "cum_var_PC2": float(cum_explained[1]) if N_PCS >= 2 else None,
    "cum_var_PC5": float(cum_explained[4]) if N_PCS >= 5 else None,
    "cum_var_PC10": float(cum_explained[9]) if N_PCS >= 10 else None,
    "cum_var_PC20": float(cum_explained[19]) if N_PCS >= 20 else None,
}

print("PCA variance fingerprint:")
for k, v in PCA_FINGERPRINT.items():
    print(f"  {k:>14s}: {v}")

# ------------------------
# Extract leading direction (PC1 axis in gene space)
# ------------------------
v1 = pca.components_[0].astype(np.float64, copy=True)
v1 /= np.linalg.norm(v1)

# PC1 scores via explicit projection (sanity vs pc_df["PC1"])
pc1_scores = delta_matrix @ v1
rho_check = sp.stats.spearmanr(pc1_scores, pc_df["PC1"].values).statistic
assert np.isfinite(rho_check) and abs(rho_check) > 0.999999, "PC1 score mismatch vs PCA transform"

print(f"\nPC1 score sanity (Spearman with PCA PC1): {rho_check:.6f}")
print("\nCell 8 complete. PCA fit + PC1 extracted. No external labels used.")


PCA variance fingerprint:
  n_perturbations: 122
         n_genes: 21592
    n_pcs_fitted: 50
         var_PC1: 0.36763845402452544
     cum_var_PC2: 0.420393442805948
     cum_var_PC5: 0.5166831540505475
    cum_var_PC10: 0.6004512392854652
    cum_var_PC20: 0.6963495155295182

PC1 score sanity (Spearman with PCA PC1): 1.000000

Cell 8 complete. PCA fit + PC1 extracted. No external labels used.


In [9]:
# ============================================================
# v6 — Cell 9: PC1 Dominance Is Necessary (Variance + Concentration)
# ============================================================
# Purpose:
#   Formalize Claim 1:
#     "PC1 dominates variance in perturbation deltas."
#
# We report:
#   - variance explained by PC1 (global)
#   - per-perturbation fraction explained by PC1 (vector decomposition)
#   - identify perturbations where PC1 explains very little / a lot
#
# Scope:
#   - Pure geometry (no biology, no external labels)
# ============================================================

assert "delta_matrix" in globals(), "delta_matrix missing"
assert "v1" in globals(), "v1 missing (run Cell 8)"
assert np.isclose(np.linalg.norm(v1), 1.0, atol=1e-8), "v1 not unit-norm"

# ------------------------
# Decompose each Δ into parallel (PC1) + orthogonal residual
# ------------------------
pc1_scores = delta_matrix @ v1                           # (n_perts,)
parallel_component = np.outer(pc1_scores, v1)            # (n_perts × n_genes)
residual_matrix = delta_matrix - parallel_component

# Norms
delta_norms = np.linalg.norm(delta_matrix, axis=1)
parallel_norms = np.linalg.norm(parallel_component, axis=1)
residual_norms = np.linalg.norm(residual_matrix, axis=1)

# Exactness check
recon_err = np.linalg.norm(delta_matrix - (parallel_component + residual_matrix))
assert np.isclose(recon_err, 0.0, atol=1e-8), "Decomposition not exact"

# Fraction explained by PC1 per perturbation (using norms)
fraction_explained_by_pc1 = np.divide(
    parallel_norms,
    delta_norms,
    out=np.zeros_like(parallel_norms),
    where=delta_norms > 0,
)

# ------------------------
# Summary fingerprint (paper-ready)
# ------------------------
PC1_DOMINANCE_FINGERPRINT = {
    "var_explained_PC1_global": float(pca.explained_variance_ratio_[0]),
    "fraction_explained_mean": float(fraction_explained_by_pc1.mean()),
    "fraction_explained_median": float(np.median(fraction_explained_by_pc1)),
    "fraction_explained_p10": float(np.quantile(fraction_explained_by_pc1, 0.10)),
    "fraction_explained_p90": float(np.quantile(fraction_explained_by_pc1, 0.90)),
    "fraction_explained_min": float(fraction_explained_by_pc1.min()),
    "fraction_explained_max": float(fraction_explained_by_pc1.max()),
}

print("PC1 dominance fingerprint:")
for k, v in PC1_DOMINANCE_FINGERPRINT.items():
    print(f"  {k:>26s}: {v}")

# ------------------------
# Perturbation-level table
# ------------------------
dominance_df = pd.DataFrame(
    {
        "pc1_score": pc1_scores,
        "delta_norm": delta_norms,
        "pc1_parallel_norm": parallel_norms,
        "pc1_residual_norm": residual_norms,
        "fraction_explained_by_pc1": fraction_explained_by_pc1,
    },
    index=pd.Index(PERTURBATION_GENES, name="perturbation"),
)

print("\nTop 10 perturbations by fraction explained (PC1 fits best):")
display(dominance_df.sort_values("fraction_explained_by_pc1", ascending=False).head(10))

print("\nBottom 10 perturbations by fraction explained (PC1 fits worst):")
display(dominance_df.sort_values("fraction_explained_by_pc1", ascending=True).head(10))

print("\nCell 9 complete. PC1 dominance quantified (global + per-perturbation).")


PC1 dominance fingerprint:
    var_explained_PC1_global: 0.36763845402452544
     fraction_explained_mean: 0.3961941117146197
   fraction_explained_median: 0.40647545669646445
      fraction_explained_p10: 0.0881075600401139
      fraction_explained_p90: 0.7087497839199183
      fraction_explained_min: 0.013439956341175993
      fraction_explained_max: 0.9240362045188799

Top 10 perturbations by fraction explained (PC1 fits best):


,pc1_score,delta_norm,pc1_parallel_norm,pc1_residual_norm,fraction_explained_by_pc1
perturbation,,,,,
RNASEH2C,24.115191,26.097669,24.115191,9.977269,0.924036
FAM136A,17.327424,19.110749,17.327424,8.061087,0.906685
NFIB,8.190636,9.518706,8.190636,4.849663,0.860478
SMAD2,8.096443,9.461589,8.096443,4.895842,0.855717
EP400,13.410413,16.369464,13.410413,9.387234,0.819233
ETV1,-6.697639,8.561182,6.697639,5.332491,0.782326
CEBPA,-10.572580,13.524040,10.572580,8.433280,0.781762
PLAGL1,5.285510,6.917456,5.285510,4.462576,0.764083
FOS,4.222337,5.719361,4.222337,3.857845,0.738253



Bottom 10 perturbations by fraction explained (PC1 fits worst):


,pc1_score,delta_norm,pc1_parallel_norm,pc1_residual_norm,fraction_explained_by_pc1
perturbation,,,,,
PBX1,-0.092247,6.863661,0.092247,6.863042,0.013440
SOX13,0.111310,6.239589,0.111310,6.238596,0.017839
PDLIM4,0.115412,6.099744,0.115412,6.098652,0.018921
HMGN3,0.159822,7.075707,0.159822,7.073902,0.022587
BCL6,-0.140279,6.069378,0.140279,6.067756,0.023113
RUNX1,-0.147458,5.131759,0.147458,5.129640,0.028734
HDAC2,0.200432,6.231908,0.200432,6.228684,0.032162
SUPT5H,-0.315373,9.244723,0.315373,9.239342,0.034114
PDCD11,-0.363731,6.502653,0.363731,6.492473,0.055936



Cell 9 complete. PC1 dominance quantified (global + per-perturbation).


In [10]:
# ============================================================
# v6 — Cell 10: Alignment With Adipogenic Priming (External-Only)
# ============================================================
# Purpose (Section 4 of v6):
#   Formalize Claim 2 (necessity-style evidence):
#     "PC1 aligns with adipogenic priming / differentiation gradients."
#
# Constraints:
#   - External labels are evaluation-only (no construction, no tuning)
#   - Report rank-based correlations (Spearman) + p-values
#   - Keep this strictly numeric (no biological interpretation)
#
# Outputs:
#   - program_props (aligned)
#   - alignment_df (correlation table)
# ============================================================

from scipy.stats import spearmanr

assert "pc1_scores" in globals(), "pc1_scores missing (run Cell 9 or Cell 8)"
assert "PERTURBATION_GENES" in globals(), "PERTURBATION_GENES missing"
assert (DATA_DIR / "program_proportion.csv").exists(), "program_proportion.csv missing"

# ------------------------
# Load external program proportions (evaluation-only)
# ------------------------
PROGRAM_PATH = DATA_DIR / "program_proportion.csv"
program_props = pd.read_csv(PROGRAM_PATH, index_col=0)

# Align to perturbation ordering exactly
program_props = program_props.loc[list(PERTURBATION_GENES)]

assert program_props.shape[0] == len(PERTURBATION_GENES), "Program proportion alignment failed"
assert np.all(np.isfinite(program_props.to_numpy())), "Non-finite values in program proportions"

# ------------------------
# Correlation table (PC1 vs each program)
# ------------------------
rows = []
for col in program_props.columns:
    rho, p = spearmanr(pc1_scores, program_props[col].values)
    rows.append(
        {
            "program": col,
            "spearman_rho": float(rho),
            "abs_rho": float(abs(rho)),
            "p_value": float(p),
        }
    )

alignment_df = (
    pd.DataFrame(rows)
      .sort_values("abs_rho", ascending=False)
      .reset_index(drop=True)
)

print("PC1 alignment with external programs (Spearman):")
display(alignment_df)

# ------------------------
# Minimal fingerprint (for Results)
# ------------------------
if "adipo" in program_props.columns:
    rho_adipo, p_adipo = spearmanr(pc1_scores, program_props["adipo"].values)
    ALIGNMENT_FINGERPRINT = {
        "rho_pc1_vs_adipo": float(rho_adipo),
        "p_pc1_vs_adipo": float(p_adipo),
        "n_perturbations": int(len(PERTURBATION_GENES)),
    }
    print("\nAlignment fingerprint (key axis):")
    for k, v in ALIGNMENT_FINGERPRINT.items():
        print(f"  {k:>18s}: {v}")

print("\nCell 10 complete. External alignment quantified (evaluation-only).")


PC1 alignment with external programs (Spearman):


,program,spearman_rho,abs_rho,p_value
0,adipo,0.965721,0.965721,3.883900e-72
1,lipo,0.887789,0.887789,2.962276e-42
2,pre_adipo,-0.854795,0.854795,5.557562e-36
3,lipo_adipo,0.528592,0.528592,3.898470e-10
4,other,-0.387247,0.387247,1.050822e-05



Alignment fingerprint (key axis):
    rho_pc1_vs_adipo: 0.9657210929213287
      p_pc1_vs_adipo: 3.883899654166383e-72
     n_perturbations: 122

Cell 10 complete. External alignment quantified (evaluation-only).


In [11]:
# ============================================================
# v6 — Cell 11: Null Model 1 (Random-Axis Null)
# ============================================================
# Purpose (Section 6 of v6):
#   Test whether the observed PC1 alignment (with an external scalar)
#   is exceptional relative to arbitrary directions in gene space.
#
# Null:
#   Random unit vectors v ~ N(0, I) / ||.||  (sign-invariant statistic)
#
# Test statistic:
#   |Spearman( Δ_p · v , adipo_prop )|
#
# Outputs:
#   - null_abs_rhos (array)
#   - empirical p-value for PC1 vs null
#
# Scope:
#   - External labels used strictly for evaluation
#   - No refitting of PC1
# ============================================================

from scipy.stats import spearmanr

assert "delta_matrix" in globals(), "delta_matrix missing"
assert "pc1_scores" in globals(), "pc1_scores missing"
assert "program_props" in globals(), "program_props missing (run Cell 10)"
assert "adipo" in program_props.columns, "Required column 'adipo' missing"

adipo_prop = program_props["adipo"].values
assert np.all(np.isfinite(adipo_prop)), "Non-finite adipo proportions"

# Observed statistic (PC1)
rho_pc1, _ = spearmanr(pc1_scores, adipo_prop)
rho_pc1_abs = abs(float(rho_pc1))

# Null sampling
N_RANDOM = 1000
rng = np.random.default_rng(RANDOM_STATE)

n_perts, n_genes = delta_matrix.shape
null_abs_rhos = np.empty(N_RANDOM, dtype=np.float64)

for i in range(N_RANDOM):
    v_rand = rng.normal(size=n_genes)
    v_rand /= np.linalg.norm(v_rand)

    scores_rand = delta_matrix @ v_rand
    rho_rand, _ = spearmanr(scores_rand, adipo_prop)
    null_abs_rhos[i] = abs(float(rho_rand))

# Empirical significance (one-sided: how often null >= observed)
p_empirical = float(np.mean(null_abs_rhos >= rho_pc1_abs))

NULL1_FINGERPRINT = {
    "n_random_axes": int(N_RANDOM),
    "null_mean_abs_rho": float(null_abs_rhos.mean()),
    "null_p95_abs_rho": float(np.percentile(null_abs_rhos, 95)),
    "observed_abs_rho_pc1": float(rho_pc1_abs),
    "empirical_p_value": float(p_empirical),
}

print("Random-axis null (|Spearman rho|):")
for k, v in NULL1_FINGERPRINT.items():
    print(f"  {k:>22s}: {v}")

print("\nCell 11 complete. Random-axis null evaluated (PC1 is compared to arbitrary directions).")


Random-axis null (|Spearman rho|):
           n_random_axes: 1000
       null_mean_abs_rho: 0.38763371285210396
        null_p95_abs_rho: 0.752655718962565
    observed_abs_rho_pc1: 0.9657210929213287
       empirical_p_value: 0.0

Cell 11 complete. Random-axis null evaluated (PC1 is compared to arbitrary directions).


In [12]:
# ============================================================
# v6 — Cell 12: Null Model 2 (Shuffled-Label Null)
# ============================================================
# Purpose (Section 6 of v6):
#   Test whether PC1 alignment could arise purely from chance pairing
#   between perturbations and external program proportions.
#
# Null:
#   Shuffle adipo labels across perturbations (break correspondence)
#
# Test statistic:
#   |Spearman( PC1_score , shuffled_adipo )|
#
# Outputs:
#   - null_shuffle_abs_rhos (array)
#   - empirical p-value for observed PC1 alignment
#
# Scope:
#   - PC1 is fixed (no refit)
#   - External labels used strictly for evaluation
# ============================================================

from scipy.stats import spearmanr

assert "pc1_scores" in globals(), "pc1_scores missing"
assert "adipo_prop" in globals(), "adipo_prop missing (Cell 11)"
assert len(pc1_scores) == len(adipo_prop), "Length mismatch PC1 vs adipo"

N_SHUFFLES = 2000
rng = np.random.default_rng(RANDOM_STATE)

null_shuffle_abs_rhos = np.empty(N_SHUFFLES, dtype=np.float64)

for i in range(N_SHUFFLES):
    shuffled = rng.permutation(adipo_prop)
    rho, _ = spearmanr(pc1_scores, shuffled)
    null_shuffle_abs_rhos[i] = abs(float(rho))

rho_pc1, _ = spearmanr(pc1_scores, adipo_prop)
rho_pc1_abs = abs(float(rho_pc1))

p_empirical_shuffle = float(np.mean(null_shuffle_abs_rhos >= rho_pc1_abs))

NULL2_FINGERPRINT = {
    "n_shuffles": int(N_SHUFFLES),
    "null_mean_abs_rho": float(null_shuffle_abs_rhos.mean()),
    "null_p95_abs_rho": float(np.percentile(null_shuffle_abs_rhos, 95)),
    "observed_abs_rho_pc1": float(rho_pc1_abs),
    "empirical_p_value": float(p_empirical_shuffle),
}

print("Shuffled-label null (|Spearman rho|):")
for k, v in NULL2_FINGERPRINT.items():
    print(f"  {k:>22s}: {v}")

print("\nCell 12 complete. Shuffled-label null evaluated (PC1 vs chance label pairing).")


Shuffled-label null (|Spearman rho|):
              n_shuffles: 2000
       null_mean_abs_rho: 0.07014963460845837
        null_p95_abs_rho: 0.17249591485068347
    observed_abs_rho_pc1: 0.9657210929213287
       empirical_p_value: 0.0

Cell 12 complete. Shuffled-label null evaluated (PC1 vs chance label pairing).


In [13]:
# ============================================================
# v6 — Cell 13: Insufficiency Test (PC1 Not Sufficient)
# ============================================================
# Purpose (Section 5 of v6):
#   Formalize Claim 3/4:
#     PC1 is necessary but insufficient once residual structure is respected.
#
# We implement a falsifiable sufficiency test:
#   - Bin perturbations by PC1 (quantile bins; fixed rule)
#   - Within each bin, test whether adipo outcomes still vary systematically
#     with PC1-orthogonal residual magnitude.
#
# Null (PC1 suffices):
#   Within PC1 bins, adipo is independent of residual magnitude.
#
# Test statistic:
#   Spearman correlation between residual_norm and adipo within bins,
#   aggregated across bins (meta-style: count significant bins + combined p)
#
# Outputs:
#   - insuff_df: per-perturbation table with PC1, residuals, adipo, bins
#   - bin_stats: per-bin correlation results
#   - global permutation p-value for the bin-aggregated statistic
# ============================================================

from scipy.stats import spearmanr

assert "dominance_df" in globals(), "dominance_df missing (run Cell 9)"
assert "program_props" in globals(), "program_props missing (run Cell 10)"
assert "adipo" in program_props.columns, "adipo missing in program_props"

# ------------------------
# Assemble analysis table
# ------------------------
insuff_df = dominance_df.copy()
insuff_df["adipo"] = program_props["adipo"].values
insuff_df = insuff_df.reset_index()  # keep perturbation name as column

# Fixed binning rule (pre-committed)
N_BINS = 5
insuff_df["pc1_bin"] = pd.qcut(insuff_df["pc1_score"], q=N_BINS, labels=False, duplicates="drop")

# ------------------------
# Per-bin residual dependence test
# ------------------------
bin_rows = []
for b in sorted(insuff_df["pc1_bin"].unique()):
    sub = insuff_df[insuff_df["pc1_bin"] == b].copy()
    assert sub.shape[0] > 5, "Bin too small for correlation test"

    rho, p = spearmanr(sub["pc1_residual_norm"].values, sub["adipo"].values)
    bin_rows.append(
        {
            "pc1_bin": int(b),
            "n_perts": int(sub.shape[0]),
            "spearman_rho(residual,adipo)": float(rho),
            "p_value": float(p),
        }
    )

bin_stats = pd.DataFrame(bin_rows).sort_values("pc1_bin")

print("Within-PC1-bin residual dependence (Spearman):")
display(bin_stats)

# ------------------------
# Aggregate statistic (simple, explicit)
# ------------------------
# Count how many bins show positive association (direction-free alternative is possible,
# but we keep this transparent; reviewers can see the table).
alpha = 0.05
n_sig = int(np.sum(bin_stats["p_value"] < alpha))

# Aggregate strength: mean absolute rho across bins
mean_abs_rho = float(np.mean(np.abs(bin_stats["spearman_rho(residual,adipo)"].values)))

AGG_FINGERPRINT = {
    "n_bins": int(bin_stats.shape[0]),
    "alpha": float(alpha),
    "n_bins_significant": int(n_sig),
    "mean_abs_rho_across_bins": float(mean_abs_rho),
}

print("\nAggregated insufficiency summary:")
for k, v in AGG_FINGERPRINT.items():
    print(f"  {k:>28s}: {v}")

# ------------------------
# Permutation test for the aggregate statistic (falsification-style)
# ------------------------
# Null: within each PC1 bin, adipo labels are exchangeable (PC1-only sufficiency).
# We permute adipo *within bins* and recompute mean_abs_rho.
N_PERM = 2000
rng = np.random.default_rng(RANDOM_STATE)

perm_stats = np.empty(N_PERM, dtype=np.float64)

for i in range(N_PERM):
    rhos = []
    for b in sorted(insuff_df["pc1_bin"].unique()):
        sub = insuff_df[insuff_df["pc1_bin"] == b]
        y_perm = rng.permutation(sub["adipo"].values)
        rho, _ = spearmanr(sub["pc1_residual_norm"].values, y_perm)
        rhos.append(abs(float(rho)))
    perm_stats[i] = float(np.mean(rhos))

p_perm = float(np.mean(perm_stats >= mean_abs_rho))

print(f"\nPermutation p-value (bin-wise exchangeability null): {p_perm:.4f}")
print("\nCell 13 complete. Sufficiency null tested using residual-aware, outcome-conditioned structure.")


Within-PC1-bin residual dependence (Spearman):


,pc1_bin,n_perts,"spearman_rho(residual,adipo)",p_value
0,0,25,-0.402308,0.046184
1,1,24,0.254783,0.229559
2,2,24,-0.327826,0.117852
3,3,24,-0.135652,0.527379
4,4,25,0.456154,0.021914



Aggregated insufficiency summary:
                        n_bins: 5
                         alpha: 0.05
            n_bins_significant: 2
      mean_abs_rho_across_bins: 0.3153444816053511

Permutation p-value (bin-wise exchangeability null): 0.0115

Cell 13 complete. Sufficiency null tested using residual-aware, outcome-conditioned structure.


In [14]:
# ============================================================
# v6 — Cell 14: Failure Modes (Same PC1, Divergent Outcomes)
# ============================================================
# Purpose (Section 5 of v6):
#   Make insufficiency concrete by enumerating "PC1-collisions":
#     perturbation pairs with similar PC1 but large outcome divergence.
#
# This is NOT interpretation. It is an explicit failure-case listing:
#   - Similar PC1 score (within tight tolerance)
#   - Large |Δ adipo| difference
#   - Optionally prioritize large residual norms (orthogonal structure)
#
# Outputs:
#   - collisions_df: top failure-case pairs
# ============================================================

assert "insuff_df" in globals(), "insuff_df missing (run Cell 13)"
assert "dominance_df" in globals(), "dominance_df missing (run Cell 9)"

# Build a compact per-perturbation table
P = (
    insuff_df[["perturbation", "pc1_score", "pc1_residual_norm", "adipo"]]
    .copy()
    .set_index("perturbation")
)

# ------------------------
# Define "same PC1" tolerance (fixed, pre-committed)
# ------------------------
# Use a robust scale of PC1 to define closeness:
pc1_vals = P["pc1_score"].values
pc1_iqr = np.quantile(pc1_vals, 0.75) - np.quantile(pc1_vals, 0.25)
PC1_TOL = 0.05 * pc1_iqr  # tight: 5% of IQR

assert PC1_TOL > 0, "Degenerate PC1 tolerance"

# ------------------------
# Enumerate collision pairs (O(n^2) is fine for n=122)
# ------------------------
perts = P.index.to_list()
rows = []

for i in range(len(perts)):
    for j in range(i + 1, len(perts)):
        a, b = perts[i], perts[j]

        pc1_a, pc1_b = P.loc[a, "pc1_score"], P.loc[b, "pc1_score"]
        if abs(pc1_a - pc1_b) > PC1_TOL:
            continue

        adipo_a, adipo_b = P.loc[a, "adipo"], P.loc[b, "adipo"]
        delta_adipo = abs(adipo_a - adipo_b)

        # Residual emphasis (mean residual magnitude of the pair)
        res_mean = 0.5 * (P.loc[a, "pc1_residual_norm"] + P.loc[b, "pc1_residual_norm"])

        rows.append(
            {
                "perturbation_A": a,
                "perturbation_B": b,
                "pc1_A": float(pc1_a),
                "pc1_B": float(pc1_b),
                "abs_pc1_diff": float(abs(pc1_a - pc1_b)),
                "adipo_A": float(adipo_a),
                "adipo_B": float(adipo_b),
                "abs_adipo_diff": float(delta_adipo),
                "mean_residual_norm": float(res_mean),
            }
        )

collisions_df = pd.DataFrame(rows)

if collisions_df.shape[0] == 0:
    print("No PC1-collision pairs found under the current tolerance.")
else:
    # Rank by outcome divergence, then by residual magnitude
    collisions_df = (
        collisions_df
        .sort_values(["abs_adipo_diff", "mean_residual_norm"], ascending=False)
        .reset_index(drop=True)
    )

    print(f"PC1 collision tolerance: {PC1_TOL:.6f} (5% of PC1 IQR)")
    print(f"Number of collision pairs found: {collisions_df.shape[0]}")

    print("\nTop 20: Similar PC1 but divergent adipo outcomes")
    display(collisions_df.head(20))

print("\nCell 14 complete. Failure-mode pairs enumerated (same PC1, divergent outcomes).")


PC1 collision tolerance: 0.250312 (5% of PC1 IQR)
Number of collision pairs found: 262

Top 20: Similar PC1 but divergent adipo outcomes


,perturbation_A,perturbation_B,pc1_A,pc1_B,abs_pc1_diff,adipo_A,adipo_B,abs_adipo_diff,mean_residual_norm
0,PWP1,TCFL5,-0.911543,-1.039370,0.127827,0.279646,0.219653,0.059993,6.646150
1,TCFL5,ZNF26,-1.039370,-1.259015,0.219645,0.219653,0.268331,0.048678,6.048217
2,GRB14,NRIP1,1.612758,1.646018,0.033260,0.261477,0.307087,0.045610,6.614647
3,GRB14,ZNF480,1.612758,1.750950,0.138192,0.261477,0.305405,0.043928,6.631556
4,HIF1A,PWP1,-0.930494,-0.911543,0.018951,0.237569,0.279646,0.042077,6.290791
5,PHF3,SRPK1,-6.021446,-5.997044,0.024402,0.153680,0.195122,0.041442,11.447118
6,CHAF1A,EBF2,-0.596592,-0.541757,0.054836,0.231655,0.271505,0.039851,4.921266
7,MXD1,PPARD,4.467180,4.226545,0.240634,0.332248,0.294872,0.037376,5.879922
8,PWP1,ZNF283,-0.911543,-1.097645,0.186102,0.279646,0.242902,0.036744,6.774006
9,SSB,TCFL5,-0.954083,-1.039370,0.085287,0.255814,0.219653,0.036161,5.876346



Cell 14 complete. Failure-mode pairs enumerated (same PC1, divergent outcomes).


In [15]:
# ============================================================
# v6 — Cell 15: Contextual Benchmarking (Question-Level, Bounded)
# ============================================================
# Purpose (Section 7 of v6):
#   Situate the diagnostic question answered here relative to
#   common perturbation-analysis frameworks, WITHOUT competing
#   on accuracy, prediction, or leaderboards.
#
# Rules:
#   - No models are trained here
#   - No numbers are optimized
#   - This cell exists to make the paper self-contained and
#     reviewer-legible, not to claim superiority
#
# Output:
#   - A compact, explicit mapping of questions → methods
# ============================================================

benchmark_rows = [
    {
        "framework": "Mixscape / clustering-based methods",
        "optimizes_for": "Discrete perturbation classification / cell-state separation",
        "cannot_answer": "Whether a single geometric axis suffices once residual structure is conditioned on outcome",
        "why_orthogonal_to_v6": "Operates at cell level; does not test distribution-level sufficiency of perturbation geometry",
    },
    {
        "framework": "GSFA / latent-factor models",
        "optimizes_for": "Learning latent factors explaining expression variance",
        "cannot_answer": "Whether dominant factors fully explain perturbation outcomes under explicit nulls",
        "why_orthogonal_to_v6": "Latent factors are learned to explain variance, not falsify sufficiency",
    },
    {
        "framework": "Predictive perturbation models",
        "optimizes_for": "Generalization to unseen perturbations",
        "cannot_answer": "Whether residual variance carries outcome-dependent structure",
        "why_orthogonal_to_v6": "Prediction success does not imply geometric sufficiency",
    },
    {
        "framework": "This work (v6)",
        "optimizes_for": "Diagnostic falsification of geometric sufficiency",
        "cannot_answer": "Out-of-sample prediction accuracy",
        "why_orthogonal_to_v6": "Explicitly rejects prediction as an objective",
    },
]

benchmark_df = pd.DataFrame(benchmark_rows)

print("Contextual benchmarking (question-level, bounded):")
display(benchmark_df)

print("\nCell 15 complete. Diagnostic scope contextualized without benchmark chasing.")


Contextual benchmarking (question-level, bounded):


,framework,optimizes_for,cannot_answer,why_orthogonal_to_v6
0,Mixscape / clustering-based methods,Discrete perturbation classification / cell-st...,Whether a single geometric axis suffices once ...,Operates at cell level; does not test distribu...
1,GSFA / latent-factor models,Learning latent factors explaining expression ...,Whether dominant factors fully explain perturb...,Latent factors are learned to explain variance...
2,Predictive perturbation models,Generalization to unseen perturbations,Whether residual variance carries outcome-depe...,Prediction success does not imply geometric su...
3,This work (v6),Diagnostic falsification of geometric sufficiency,Out-of-sample prediction accuracy,Explicitly rejects prediction as an objective



Cell 15 complete. Diagnostic scope contextualized without benchmark chasing.


In [16]:
# ============================================================
# v6 — Cell 16: Explicit Falsifiability Statement + Verdict
# ============================================================
# Purpose (Section 8 of v6):
#   Make the paper falsifiable in an explicit, audit-ready way.
#
# This cell:
#   - States the sufficiency null formally
#   - Lists the empirical consequences that MUST hold if the null were true
#   - Verifies which consequences are violated by the observed data
#
# This is not interpretation. This is a logical checklist.
# ============================================================

# ------------------------
# Formal null (PC1 sufficiency)
# ------------------------
FALSIFIABILITY_NULL = (
    "If perturbation responses were fully explainable by a single geometric axis (PC1), "
    "then after conditioning on PC1:\n"
    "  (i) residual variance would be noise-like,\n"
    "  (ii) residual magnitude would be outcome-independent,\n"
    "  (iii) perturbations with similar PC1 scores would exhibit similar outcomes."
)

print("Sufficiency null (H0):")
print(FALSIFIABILITY_NULL)

# ------------------------
# Empirical checks (binary, pre-specified)
# ------------------------
checks = []

# Check 1: Residual-outcome dependence within PC1 bins
checks.append(
    {
        "check": "Residual–outcome dependence within PC1 bins",
        "expected_under_H0": "No systematic dependence",
        "observed": f"{AGG_FINGERPRINT['n_bins_significant']} / {AGG_FINGERPRINT['n_bins']} bins significant",
        "result": "VIOLATED" if AGG_FINGERPRINT["n_bins_significant"] > 0 else "NOT VIOLATED",
    }
)

# Check 2: Aggregate permutation test
checks.append(
    {
        "check": "Bin-wise exchangeability permutation test",
        "expected_under_H0": "Non-significant",
        "observed": f"p = {p_perm:.4f}",
        "result": "VIOLATED" if p_perm < 0.05 else "NOT VIOLATED",
    }
)

# Check 3: PC1 collision existence
checks.append(
    {
        "check": "Existence of PC1-collision pairs",
        "expected_under_H0": "Rare or absent",
        "observed": f"{collisions_df.shape[0]} collision pairs found",
        "result": "VIOLATED" if collisions_df.shape[0] > 0 else "NOT VIOLATED",
    }
)

falsification_df = pd.DataFrame(checks)

print("\nFalsifiability checklist:")
display(falsification_df)

# ------------------------
# Final diagnostic verdict (mechanical)
# ------------------------
violations = falsification_df["result"].tolist().count("VIOLATED")

VERDICT = (
    "REJECT PC1 SUFFICIENCY"
    if violations > 0
    else "FAIL TO REJECT PC1 SUFFICIENCY"
)

print("\nFinal diagnostic verdict:")
print(f"  {VERDICT}")

print("\nCell 16 complete. Falsifiability explicitly stated and evaluated.")


Sufficiency null (H0):
If perturbation responses were fully explainable by a single geometric axis (PC1), then after conditioning on PC1:
  (i) residual variance would be noise-like,
  (ii) residual magnitude would be outcome-independent,
  (iii) perturbations with similar PC1 scores would exhibit similar outcomes.

Falsifiability checklist:


,check,expected_under_H0,observed,result
0,Residual–outcome dependence within PC1 bins,No systematic dependence,2 / 5 bins significant,VIOLATED
1,Bin-wise exchangeability permutation test,Non-significant,p = 0.0115,VIOLATED
2,Existence of PC1-collision pairs,Rare or absent,262 collision pairs found,VIOLATED



Final diagnostic verdict:
  REJECT PC1 SUFFICIENCY

Cell 16 complete. Falsifiability explicitly stated and evaluated.


In [17]:
# ============================================================
# v6 — Cell 17: Notebook Closure + Reproducibility Artifacts
# ============================================================
# Purpose:
#   - Freeze the v6 notebook as a complete, self-contained diagnostic
#   - Persist all first-class artifacts needed for paper figures/tables
#   - Explicitly mark analysis as COMPLETE (no hidden steps)
#
# This cell performs NO new analysis.
# It only serializes results and records provenance.
# ============================================================

from datetime import datetime
import json

# ------------------------
# Timestamp + run metadata
# ------------------------
RUN_METADATA = {
    "v6_spec": "Distribution-Aware Perturbation Geometry",
    "run_timestamp_utc": datetime.utcnow().isoformat(timespec="seconds"),
    "random_state": RANDOM_STATE,
    "n_cells": int(adata.n_obs),
    "n_genes": int(adata.n_vars),
    "n_perturbations_excl_control": len(PERTURBATION_GENES),
    "control_label": CONTROL_LABEL,
    "verdict": VERDICT,
}

# ------------------------
# Persist core tables (paper-grade)
# ------------------------
ARTIFACTS = {
    "delta_table": delta_df,
    "pc_scores": pc_df,
    "pc1_dominance": dominance_df,
    "exceedance_rates": exceed_df,
    "alignment_table": alignment_df,
    "bin_residual_stats": bin_stats,
    "pc1_collisions": collisions_df,
    "falsifiability_checklist": falsification_df,
}

for name, df in ARTIFACTS.items():
    out_path = RESULTS_DIR / f"{name}.csv"
    df.to_csv(out_path)
    print(f"Saved: {out_path}")

# ------------------------
# Persist control distribution parameters
# ------------------------
np.save(RESULTS_DIR / "control_mean.npy", control_mean)
np.save(RESULTS_DIR / "control_sd.npy", control_sd)
np.save(RESULTS_DIR / "control_q_lo.npy", control_q_lo)
np.save(RESULTS_DIR / "control_q_hi.npy", control_q_hi)
np.save(RESULTS_DIR / "control_iqr.npy", control_iqr)

print("Saved control distribution reference arrays.")

# ------------------------
# Persist null distributions
# ------------------------
np.save(RESULTS_DIR / "null_random_axis_abs_rhos.npy", null_abs_rhos)
np.save(RESULTS_DIR / "null_shuffle_label_abs_rhos.npy", null_shuffle_abs_rhos)

print("Saved null-model distributions.")

# ------------------------
# Persist run metadata
# ------------------------
with open(RESULTS_DIR / "run_metadata.json", "w") as f:
    json.dump(RUN_METADATA, f, indent=2)

print("Saved run metadata.")

# ------------------------
# Final completion statement
# ------------------------
print("\n============================================================")
print("v6 NOTEBOOK COMPLETE")
print("------------------------------------------------------------")
print("All claims are supported by explicit tests.")
print("All nulls are evaluated.")
print("All figures and tables are reproducible from saved artifacts.")
print("No predictive modeling performed.")
print("No leaderboard optimization performed.")
print(f"FINAL VERDICT: {VERDICT}")
print("============================================================")


Saved: c:\Users\Bryan\Documents\CrunchDAO Obesity\results_v6\delta_table.csv
Saved: c:\Users\Bryan\Documents\CrunchDAO Obesity\results_v6\pc_scores.csv
Saved: c:\Users\Bryan\Documents\CrunchDAO Obesity\results_v6\pc1_dominance.csv
Saved: c:\Users\Bryan\Documents\CrunchDAO Obesity\results_v6\exceedance_rates.csv
Saved: c:\Users\Bryan\Documents\CrunchDAO Obesity\results_v6\alignment_table.csv
Saved: c:\Users\Bryan\Documents\CrunchDAO Obesity\results_v6\bin_residual_stats.csv
Saved: c:\Users\Bryan\Documents\CrunchDAO Obesity\results_v6\pc1_collisions.csv
Saved: c:\Users\Bryan\Documents\CrunchDAO Obesity\results_v6\falsifiability_checklist.csv
Saved control distribution reference arrays.
Saved null-model distributions.
Saved run metadata.

v6 NOTEBOOK COMPLETE
------------------------------------------------------------
All claims are supported by explicit tests.
All nulls are evaluated.
All figures and tables are reproducible from saved artifacts.
No predictive modeling performed.
No lead